## Dynamic Routing

In [1]:
from langchain import PromptTemplate

In [2]:
classifier_template = PromptTemplate(
    input_variables=["question", "answer"],
    template="""
You are given a question and the user's answer to that question.  Classify the user's answer as either 'Correct' or 'Incorrect'.
Do not respond with more than one word.

Question: {question}
Answer: {answer}"""
)

In [3]:
from langchain_openai import ChatOpenAI

In [4]:
llm = ChatOpenAI()

In [5]:
from langchain_core.output_parsers import StrOutputParser

In [6]:
output_parser = StrOutputParser()

In [7]:
classifier_chain = classifier_template | llm | output_parser

In [8]:
classifier_chain.invoke({"question": "What color is the sky?", "answer": "blue"})

'Correct'

In [9]:
correct_template = PromptTemplate(
    input_variables=["question"],
    template="""
The user was asked the following question and answered it correctly.  Ask a more difficult question on the same topic.

Question: {question}
New Question:"""
)

In [10]:
correct_chain = correct_template | llm | output_parser

In [11]:
incorrect_template = PromptTemplate(
    input_variables=["question"],
    template="""
The user was asked the following question and answered it incorrectly.  Tell them the correct answer and explain why.

Question: {question}
Correct Answer:
Explanation:"""
)

In [12]:
incorrect_chain = incorrect_template | llm | output_parser

In [14]:
def route(info):
    if info["result"].lower() == "correct":
        return correct_chain
    elif info["result"].lower() == "incorrect":
        return incorrect_chain
    else:
        return "Format is not correct"

In [17]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda

In [21]:
final_chain = RunnableParallel(
    {
        "result": classifier_chain, 
        "question": lambda x: x["question"], 
        "answer": lambda x: x["answer"]
    }
) | RunnableParallel(
    {
        "response": RunnableLambda(route), 
        "input": RunnablePassthrough()
    }
)

In [23]:
final_chain.invoke({"question": "What color is the sky?", "answer": "pink"})

{'response': "The correct answer is blue. The sky appears blue during the day due to the way the Earth's atmosphere scatters sunlight. The shorter blue wavelengths are scattered in all directions by the gases and particles in the atmosphere, giving the sky its blue appearance.",
 'input': {'result': 'Incorrect',
  'question': 'What color is the sky?',
  'answer': 'pink'}}